<h1>Equal-Weight S&P 500 Index Fund</h1>
<h2>Introduction and Library Imports</h2>


<h2>Library Imports</h2>


In [1]:
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter
import math

<h2>Importing Our List of Stocks</h2>


In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

<h2>Acquiring an API Token</h2>

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

<h2>Making our first API Call</h2>

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

<h2>Parsing our api call</h2>


In [5]:
price = data['latestPrice']
market_cap = data['marketCap']

<h2>Adding our Stocks Data to a pandas dataframe</h2>

In [6]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)


In [7]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index=True
)


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,133.31,2285750653795,N/A


<h2>Looping through the tickers in our list of stcocks</h2>

<h2>Using Batch API calls to improve performance</h2>

In [8]:
def chunks(lst,n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [19]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_string = []
for i in range(0, len(symbol_groups)):
    symbol_string.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns= my_columns)

for symbol_string in symbol_string:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ], 
                index = my_columns
            ),ignore_index=True
        )

In [20]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,135.870,41121860568,N/A
1,AAL,23.061,15417864718,N/A
2,AAP,195.550,12603728508,N/A
3,AAPL,131.950,2296963734930,N/A
4,ABBV,113.000,198450816901,N/A
...,...,...,...,...
500,YUM,116.760,35723061207,N/A
501,ZBH,173.510,36541330735,N/A
502,ZBRA,505.900,27519620431,N/A
503,ZION,57.800,9444979570,N/A


<h2>Calculatting the number of shares to buy</h2>


In [21]:
portfolio_size = input("Enter the value of your portfolio: ")

try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print("That's not a number \n Please try again")

    portfolio_size = input("Enter the value of your portfolio: ")

    val = float(portfolio_size)

1000000.0


In [22]:
position_size = val/len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size)/final_dataframe.loc[i,'Stock Price']

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,135.870,41121860568,14.572753
1,AAL,23.061,15417864718,85.859243
2,AAP,195.550,12603728508,10.125288
3,AAPL,131.950,2296963734930,15.005684
4,ABBV,113.000,198450816901,17.522124
...,...,...,...,...
500,YUM,116.760,35723061207,16.957862
501,ZBH,173.510,36541330735,11.411446
502,ZBRA,505.900,27519620431,3.913817
503,ZION,57.800,9444979570,34.256055


<h2>Formatting our Excel Output</h2>


In [51]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine= 'xlsxwriter')
final_dataframe.to_excel(writer,'Recommended Trades', index = False)

<h2>Creating the formats we'll need for our .xlsx finle</h2>


In [52]:
backgroungColor = '#0a0a23'
fontColor = '#ffffff'

stringFormat = writer.book.add_format(
    {
        'font_color':fontColor,
        'bg_color':backgroungColor,
        'border':1
    }
)

dolarFormat = writer.book.add_format(
    {
        'num_format':'$0.00',
        'font_color':fontColor,
        'bg_color':backgroungColor,
        'border':1
    }
)

integerFormat = writer.book.add_format(
    {
        'num_format':'0',
        'font_color':fontColor,
        'bg_color':backgroungColor,
        'border':1
    }
)

<h2>Applying the Formats to the Columns of our xlsx File</h2>

In [53]:
#writer.sheets['Recommended Trades'].set_column('A:A', 18, stringFormat)
#writer.sheets['Recommended Trades'].set_column('B:B', 18, stringFormat)
#writer.sheets['Recommended Trades'].set_column('C:C', 18, stringFormat)
#writer.sheets['Recommended Trades'].set_column('D:D', 18, stringFormat)
#writer.save()

#writer.sheets['Recommended Trades'].write('A1','Ticker',stringFormat)

In [54]:
columnFormats = {
    'A':['Ticker', stringFormat],
    'B':['Stock Price', dolarFormat],
    'C':['Market Capitalization', dolarFormat],
    'D':['Number of Shares to Buy', integerFormat]
}

for column in columnFormats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, columnFormats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1',columnFormats[column][0],columnFormats[column][1])

writer.save()